<a href="https://colab.research.google.com/github/aravind598/Data_Science_Proj/blob/master/Data%20Science%20Project/Deep_Learning_EarthQuake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.1
import numpy as np
import pandas as pd

import lightgbm as lgb

import keras 
from keras.layers import *
from keras.optimizers import *
from keras.models import Model

from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold
!pip install tensorflow==1.1

     |████████████████████████████████| 31.4MB 140kB/s 
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarn

In [0]:
url = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_labels.csv?token=ANW7SFBKZIJTONPGC67MZMS6SCJR2'
urll = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_values.csv?token=ANW7SFBK2YMNTYBWW44PE7C6SCJSE'
url2 = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/test_values.csv?token=ANW7SFHJZIY2XRWW5QVSW4S6SCS5I'

In [0]:
train_x = pd.read_csv(urll)
train_y = pd.read_csv(url)
test_x  = pd.read_csv(url2)

In [0]:
geo1 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_1_id"], test_x["geo_level_1_id"]])))
geo2 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_2_id"], test_x["geo_level_2_id"]])))
geo3 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_3_id"], test_x["geo_level_3_id"]])))

In [5]:
def NET():
    inp = Input((geo3.shape[1],))
    i1 = Dense(16, name="intermediate")(inp)
    x2 = Dense(geo2.shape[1], activation='sigmoid')(i1)
    x1 = Dense(geo1.shape[1], activation='sigmoid')(i1)

    model = Model(inp, [x2,x1])
    model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
    return model

model = NET()
model.fit(geo3, [geo2, geo1], batch_size=128, epochs=10, verbose=1)

Epoch 1/10
347469/347469 [==============================] - 87s 250us/step - loss: 0.2042 - dense_1_loss: 0.0531 - dense_2_loss: 0.1512 - dense_1_acc: 0.9989 - dense_2_acc: 0.9659
Epoch 2/10
347469/347469 [==============================] - 94s 270us/step - loss: 0.0510 - dense_1_loss: 0.0051 - dense_2_loss: 0.0459 - dense_1_acc: 0.9993 - dense_2_acc: 0.9869
Epoch 3/10
347469/347469 [==============================] - 88s 254us/step - loss: 0.0143 - dense_1_loss: 0.0041 - dense_2_loss: 0.0102 - dense_1_acc: 0.9993 - dense_2_acc: 0.9977
Epoch 4/10
347469/347469 [==============================] - 88s 252us/step - loss: 0.0066 - dense_1_loss: 0.0035 - dense_2_loss: 0.0031 - dense_1_acc: 0.9993 - dense_2_acc: 0.9993
Epoch 5/10
347469/347469 [==============================] - 88s 255us/step - loss: 0.0043 - dense_1_loss: 0.0030 - dense_2_loss: 0.0013 - dense_1_acc: 0.9994 - dense_2_acc: 0.9997
Epoch 6/10
347469/347469 [==============================] - 88s 253us/step - loss: 0.0030 - dense_1_

In [0]:
# "Extract Intermediate Layer" Function
from keras import backend as K

get_int_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [0]:
# Extract GEO-Embeds for all train data points.
# Then assign with train_data

out = []
for dat in geo3[:260601]:
    layer_output = get_int_layer_output([[dat]])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

train_data = pd.get_dummies(train_x.copy())
train_data = train_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
train_data = train_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15])

In [0]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

In [11]:
y = np.array(train_y["damage_grade"])-1

df = train_data.drop(["building_id"], axis=1)
x = np.array(df)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for ix, (train_index, test_index) in enumerate(kf.split(x)):
    lgb_params = {
        "objective" : "multiclass",
        "num_class":3,
        "metric" : "multi_error",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "feature_fraction" : 0.5,
        "min_sum_hessian_in_leaf" : 0.1,
        "max_bin":8192,
        "verbosity" : 1,
        "num_threads":6,
      
    }

    x_train, x_val, y_train, y_val= x[train_index], x[test_index], y[train_index], y[test_index]

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data   = lgb.Dataset(x_val, label=y_val)

    lgb_clf = lgb.train(lgb_params,
                        train_data,
                        20000,
                        valid_sets = [val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000)

    y_pred = lgb_clf.predict(x_val)
    print("F1-MICRO SCORE: ", f1_score(np.array(pd.get_dummies(y_val)), threshold_arr(y_pred), average='micro'))
    

Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.251741
[2000]	valid_0's multi_error: 0.250436
[3000]	valid_0's multi_error: 0.250705
[4000]	valid_0's multi_error: 0.252796
[5000]	valid_0's multi_error: 0.254005
Early stopping, best iteration is:
[2327]	valid_0's multi_error: 0.249899
F1-MICRO SCORE:  0.7501007271541221
Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.250403
[2000]	valid_0's multi_error: 0.247141
[3000]	valid_0's multi_error: 0.248465
[4000]	valid_0's multi_error: 0.249731
[5000]	valid_0's multi_error: 0.250748
Early stopping, best iteration is:
[2070]	valid_0's multi_error: 0.246815
F1-MICRO SCORE:  0.7531849577897161
Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's multi_error: 0.252322
[2000]	valid_0's multi_error: 0.249885
[3000]	valid_0's multi_error: 0.249731
[4000]	valid_0's multi_error: 0.25048
[5000]	valid_0's multi_error: 0.251995
Earl